<a href="https://colab.research.google.com/github/suhyeonnnnn/DACON-Practice/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
NGROK_TOKEN = '2VHdwYwKgPdECzEwLbeortiheWl_7YXY3i1KmrmG2QmWffKp4'
PASSWORD = '9213'

In [ ]:
# gpu 확인
!nvidia-smi

Tue Sep 12 16:16:21 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install colab-ssh

In [ ]:
from colab_ssh import launch_ssh
launch_ssh(NGROK_TOKEN, PASSWORD)

we highly recommend that update your code by following this documentation https://github.com/WassimBenzarti/colab-ssh#getting-started
Successfully running 8.tcp.ngrok.io:18400
[Optional] You can also connect with VSCode SSH Remote extension using this configuration:

  Host google_colab_ssh
    HostName 8.tcp.ngrok.io
    User root
    Port 18400
    


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cd /content/drive/MyDrive/samsumgAIchallenge

In [ ]:
!ls

dataset.py  main.ipynb	models.py  __pycache__	utils.py


In [ ]:
import sys


In [ ]:
import os
import pandas as pd
import numpy as np
import cv2
from PIL import Image

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

from tqdm import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)


cuda


In [ ]:
import dataset

In [ ]:
# data loader
transform = A.Compose(
    [
        A.Resize(224, 224),
        A.Normalize(),
        ToTensorV2()
    ]
)

dataset = CustomDataset(csv_file='./train_source.csv', transform=transform)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True, num_workers=4)

NameError: ignored

In [ ]:
# model 초기화
model = UNet().to(device)

# loss function & optimizer 정의
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# training loop
for epoch in range(20): # 20 epoch 동안 학습
  model.train()
  epoch_loss = 0
  for images, masks in tqdm(dataloader):
    images = images.float().to(device)
    mask = masks.long().to(device)

    optimizer.zero_grad()
    outputs = model(images)
    loss = criterion(outputs, masks.squeeze(1))
    loss.backward()
    optimizer.step()

    epoch_loss += loss.item()

  print(f'Epoch {epoch+1}, Loss: {epoch_loss/len(dataloader)}')


In [ ]:
# infernce

In [ ]:
with torch.no_grad():
  model.eval()
  result = []

  for images in tqdm(test_dataloader):
    images = images.float().to(device)
    outputs = model(images)
    outputs = torch.softmax(outputs, dim = 1).cpu()
    outputs = torch.argmax(outputs, dim = 1).numpy()

    # batch에 존재하는 각 이미지에 대해서 반복
    for pred in outputs():
      pred = pred.astype(np.unit8)
      pred = Image.fromarray(pred) # 이미지로 변환
      pred = pred.resize((960, 540), Image.NEAREST) # 960 X 540 사이즈로 변환
      pred = np.array(pred) # 다시 수치로 변환
      # class 0 ~ 11에 해당하는 경우에 마스크 생성 (배경(12)는 제외)
      for class_id in range(12):
        class_mask = (pred==class_id).astype(np.unit8)
        if np.sum(class_mask) > 0: # 마스크가 존재하는 경우 encode
          mask_rle = rle_encode(class_mask)
          result.append(mask_rle)
        else:
          result.append(-1)


In [ ]:
# submission
submit = pd.read_csv('./sample_submission.csv')
submit['mask_rle'] = result
submit.head(3)

In [ ]:
submit.to_csv('./basline_submit.csv', index = False)